In [1]:
import json

In [10]:
import numpy as np
import scqubits as scq

# Constants
e = 1.602e-19  # elementary charge in C
hbar = 1.054e-34  # reduced Planck constant in Js
Z_0 = 50  # in Ohms

def find_g_a(C_g, C_B, f_r, Lj, N):
    C_Sigma = C_g + C_B # + 1.5e-15
    omega_r = 2 * np.pi * f_r
    EJ = ((hbar / 2 / e) ** 2) / Lj * (1.5092e24) # 1J = 1.5092e24 GHz
    EC = e**2/(2*C_Sigma) * (1.5092e24) # 1J = 1.5092e24 GHz

    transmon = scq.Transmon(EJ=EJ,
                            EC=EC,
                            ng = 0,
                            ncut = 30)

    a = transmon.anharmonicity() * 1000 # linear MHz
    g = ((C_g / C_Sigma) * omega_r * np.sqrt(N * Z_0 * e**2 / (hbar * np.pi) )* (EJ/(8*EC))**(1/4)) / 1E6 / (2 * np.pi) # linear MHz
    f_q = transmon.E01()
    
    return g, a, f_q

In [11]:
# predicted_cross_length = [211.36686180080213, 239.67001841727927, 199.29546197022296, 272.80405439803087, 166.21118594572997]
# predicted_claw_length = [62.74561331858793, 238.54614619784087, 166.65135380618355, 49.04728878924247, 122.98283288276569]
# Lj_vals = [10.130278903289723, 8.837126261668763, 10.545628251119771, 7.934490145874526, 12.130125654474945]

vec1 = (12.101812915800638, 165.60709433301977, 142.67234304675804)
vec2 = (10.092518176996967, 210.20946233643153, 201.9365897494747)
Lj_vals = []
predicted_cross_length = []
predicted_claw_length = []

for Lj, xlen, clen in [vec1, vec2]:
    Lj_vals += [Lj]
    predicted_cross_length += [xlen]
    predicted_claw_length += [clen]

In [12]:
# Qubit Identifier 1

C_g = 5.91536e-15	# in F
C_B = 122.44002e-15	# in F
f_r = 6.116e9 # linear frequency in Hz
f_q = 4.216 # Linear freq in GHz
Lj = 10e-9
anharmonicity = -0.153 # Linear freq in GHz


print('Qubit 1')
print("Experimental g = 61 Linear MHz")
# print(f"Simulated g = {g(C_g, C_B, f_r, f_q, anharmonicity, 4) / 1E6 / (2 * np.pi)} Linear MHz")
print(f"Simulated g = {find_g_a(C_g, C_B, f_r, Lj, 4)[0]} Linear MHz")
print(f"Simulated a = {find_g_a(C_g, C_B, f_r, Lj, 4)[1]} Linear MHz")
print(f"Simulated f_q = {find_g_a(C_g, C_B, f_r, Lj, 4)[2]} Linear GHz")

Qubit 1
Experimental g = 61 Linear MHz
Simulated g = 67.30512966260582 Linear MHz
Simulated a = -164.61390750785299 Linear MHz
Simulated f_q = 4.283443913871817 Linear GHz


In [18]:
g = open(f'qubitonly_{0}_v2.json')

data_g = json.load(g)

print(data_g["data"])

{'cross_to_ground': -77.99951, 'claw_to_ground': -73.91245, 'cross_to_claw': -5.39135, 'cross_to_cross': 85.29643, 'claw_to_claw': 79.85653, 'ground_to_ground': 199.00772}


In [22]:
data = {}

for ind in range(2):
    if ind != 1:
        continue
    f = open(f'cavity_claw_{ind}_v2.json')

    data_f = json.load(f)

    # for i in data_f:
    #     print(data_f[i])

    g = open(f'qubitonly_{ind}_v2.json')

    data_g = json.load(g)

    # for i in data_g["data"]:
    #     print(data_g['data'][i])
    cross2cpw = abs(data_g["data"]["cross_to_claw"]) * 1e-15
    cross2ground = 0 #abs(data_g["data"]["cross_to_ground"]) * 1e-15
    f_r = data_f["sim_results"]["cavity_frequency"]
    Lj = Lj_vals[ind]*(10**-9)
    print(Lj)
    gg, aa, ff_q = find_g_a(cross2cpw, cross2ground, f_r, Lj, N=4)
    data.update({f"qubit_{ind}": dict(
        f_cavity = f_r,
        Q = data_f["sim_results"]["Q"],
        kappa = data_f["sim_results"]["kappa"],
        g = gg,
        a = aa,
        f_qubit = ff_q
    )})
    # print(g)
    # print(a)
print(data)


1.0092518176996967e-08
{'qubit_1': {'f_cavity': 6412448932.31, 'Q': 48631.25307883098, 'kappa': 131858.5996934, 'g': 778.7079581498427, 'a': -6926.280344357792, 'f_qubit': 14.938126313107318}}


In [20]:
data

{'qubit_0': {'f_cavity': 6587470711.18,
  'Q': 45720.814341292695,
  'kappa': 144080.3451576,
  'g': 87.05121589046766,
  'a': -263.485074814815,
  'f_qubit': 4.76270018690812}}

In [78]:
from utils import *
save_simulation_data_to_json(data, "final_results_without_q1.json")